# Data Science 2020 course 
## Audio Book Deep Learning

### This is about the dataset of a book selling company

#### What to include here
1. how to subtitute average value in a column with empty spaces or null values
2. how to do the process above in excell worksheet

## Preprocess the Data

In [12]:
# import necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
sn.set()

In [28]:
# extract the dataset

dataset = np.loadtxt("1.2 Audiobooks_data.csv", delimiter=",")

# Extract the unscaled imput and the unscaled target leaving the first column
unscaled_input_all = dataset[:,1:-1]

target_all = dataset[:,-1]


In [37]:
# Balancing the dataset

num_of_ones_target = int(np.sum(target_all))
zero_target_counter = 0
input_to_remove = []


for i in range(target_all.shape[0]):
    if target_all[i]== 0:
        zero_target_counter += 1
        if zero_target_counter > num_of_ones_target:
            input_to_remove.append(i)
            
unscaled_input_prior = np.delete(unscaled_input_all, input_to_remove, axis=0)
unscaled_target_prior = np.delete(target_all, input_to_remove, axis = 0)



In [44]:
# standardising the imput

from sklearn import preprocessing 

scaled_input = preprocessing.scale(unscaled_input_prior)




In [50]:
# shuffling the data

shuffle_indices = np.arange(scaled_input.shape[0])
np.random.shuffle(shuffle_indices)


shuffled_input = scaled_input[shuffle_indices]
shuffled_target = unscaled_target_prior[shuffle_indices]

In [69]:
# split the dataset into train, test and validation 80 10 10

# take the sample count
sample_count = shuffled_input.shape[0]

train_sample_count = int(0.8 * sample_count)

validation_sample_count = int(0.1 * sample_count)

test_sample_count = sample_count - (train_sample_count + validation_sample_count)

# take the train, test and validation

train_inputs = shuffled_input[:train_sample_count]
train_targets = shuffled_target[:train_sample_count]

test_inputs = shuffled_input[:test_sample_count]
test_targets = shuffled_target[:test_sample_count]

validation_inputs = shuffled_input[:validation_sample_count]
validation_targets = shuffled_target[:validation_sample_count]

In [70]:
# save the three sample in the npz format


np.savez("Audiobooks_data_train", inputs = train_inputs, targets = train_targets)
np.savez("Audiobooks_data_test", inputs = test_inputs, targets = test_targets)
np.savez("Audiobooks_data_validation", inputs = validation_inputs, targets = validation_targets)



## Create the machine learning algorithm

In [84]:
import tensorflow as tf

### Data

In [82]:
# load the data

npz = np.load("Audiobooks_data_train.npz")

train_inputs = npz["inputs"].astype(np.float)
train_targets = npz["targets"].astype(np.int)

ayo = np.load("Audiobooks_data_test.npz")
test_inputs = ayo["inputs"].astype(np.float)
test_targets = ayo["targets"].astype(np.int)

mumu = np.load("Audiobooks_data_validation.npz")
validation_inputs = mumu["inputs"].astype(np.float)
validation_targets = mumu["targets"].astype(np.int)

### Model

Outline, Optimizer, Loss, Early stopping and Training

In [90]:
input_size = 10
output_size = 2
hidden_layer_size = 50 

model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation = "relu"),
    tf.keras.layers.Dense(hidden_layer_size, activation = "relu"),
    tf.keras.layers.Dense(output_size, activation = "softmax")
])

model.compile(optimizer = "adam", loss = "sparse_categorical_crossentropy", metrics = ["accuracy"] )

batch_size = 100
early_stopping = tf.keras.callbacks.EarlyStopping(patience = 2)
max_epochs = 100

model.fit(train_inputs,
         train_targets,
         batch_size= batch_size,
         epochs = max_epochs,
          callbacks = [early_stopping],
         validation_data = (validation_inputs, validation_targets),
         verbose = 2)

Epoch 1/100
36/36 - 0s - loss: 0.6180 - accuracy: 0.6426 - val_loss: 0.5302 - val_accuracy: 0.7383
Epoch 2/100
36/36 - 0s - loss: 0.4922 - accuracy: 0.7449 - val_loss: 0.4491 - val_accuracy: 0.7785
Epoch 3/100
36/36 - 0s - loss: 0.4302 - accuracy: 0.7712 - val_loss: 0.4205 - val_accuracy: 0.7651
Epoch 4/100
36/36 - 0s - loss: 0.4004 - accuracy: 0.7751 - val_loss: 0.3983 - val_accuracy: 0.7718
Epoch 5/100
36/36 - 0s - loss: 0.3820 - accuracy: 0.7910 - val_loss: 0.3827 - val_accuracy: 0.7875
Epoch 6/100
36/36 - 0s - loss: 0.3713 - accuracy: 0.7985 - val_loss: 0.3718 - val_accuracy: 0.7919
Epoch 7/100
36/36 - 0s - loss: 0.3612 - accuracy: 0.8069 - val_loss: 0.3654 - val_accuracy: 0.7897
Epoch 8/100
36/36 - 0s - loss: 0.3546 - accuracy: 0.8083 - val_loss: 0.3601 - val_accuracy: 0.7987
Epoch 9/100
36/36 - 0s - loss: 0.3519 - accuracy: 0.8089 - val_loss: 0.3552 - val_accuracy: 0.7942
Epoch 10/100
36/36 - 0s - loss: 0.3518 - accuracy: 0.8075 - val_loss: 0.3517 - val_accuracy: 0.7919
Epoch 11/

### Test the model


In [91]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

14/14 [==============================] - 0s 5ms/step - loss: 0.3460 - accuracy: 0.8080


In [100]:
print("\n Test loss: {0:.2f}, Test Accuracy: {1:.2f}% .".format(test_loss, test_accuracy * 100))


 Test loss: 0.35, Test Accuracy: 80.80% .
